In [17]:
from keras.utils.data_utils  import get_file
from keras import backend as K
from keras import imagenet_utils

ImportError: cannot import name 'imagenet_utils'

In [27]:
import keras
from keras.datasets import mnist
from keras.layers import Input, Embedding, LSTM, Dense, Lambda
from keras.models import Model
import numpy as np
from keras.utils import np_utils
from keras.layers.core import Dense, Dropout, Activation

nb_classes = 10

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)



from keras.models import Sequential
from keras.layers import Dense, Merge
from keras.optimizers import SGD, Adam, RMSprop

batch_size = 128
nb_classes = 10
nb_epoch = 3

teacher = Sequential()
teacher.add(Dense(10, input_shape=(784,)))
teacher.add(Dense(10))
teacher.add(Activation('softmax'))

teacher.summary()
teacher.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = teacher.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = teacher.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

for i in range(len(teacher.layers)):
    setattr(teacher.layers[i], 'trainable', False)



Y_train = np.zeros((60000, 10))


### build student model
student = Sequential()
student.add(Convolution2D(32, 3, 3, input_shape=(128, 128, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))

student.add(Convolution2D(64, 3, 3))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))

student.add(Flatten())
student.add(Dense(64))
student.add(Activation('relu'))
student.add(Dropout(0.5))
student.add(Dense(2))
student.add(Activation('softmax'))

student.summary()

student.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])


### teacher educat student
from keras.layers import Activation, Dence, Add
negativeActivation = lambda x: -x

negativeRight = Activation(negativeActivation)(student.output) 
diff = Add()([teacher.output,negativeRight])


model = Model(inputs=[teacher.input, student.input], outputs=[diff])
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['acc'])

model.summary(line_length=150)
model.fit([X_train, X_train], [Y_train], batch_size=128, nb_epoch=5)


print(student.evaluate(X_test, Y_test))

5

In [58]:
class EducateGenerator(object):
    def __init__(self, train_generator, validation_generator):
        self.train_generator = train_generator
        self.validation_generator = validation_generator
    def __next__(self):
        train_batch = self.train_generator.next()
        validation_batch = self.validation_generator.next()
        return ([train_batch,train_batch],validation_batch)
    def next(self):
        train_batch = self.train_generator.next()
        validation_batch = self.validation_generator.next()
        return ([train_batch,train_batch],validation_batch)


In [59]:
educate_generator = EducateGenerator(
    train_generator, validation_generator)

In [62]:
educate_generator.next()

AttributeError: 'list' object has no attribute 'shape'

In [56]:
data_dir = '../data/eyes/'

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)


train_generator = train_datagen.flow_from_directory(
    data_dir + 'train',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    data_dir + 'validation',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical')



Found 514 images belonging to 2 classes.
Found 56 images belonging to 2 classes.


In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32
epochs = 1


student = Sequential()
student.add(Convolution2D(32, 3, 3, input_shape=(128, 128, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))

student.add(Convolution2D(64, 3, 3))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))

student.add(Flatten())
student.add(Dense(64))
student.add(Activation('relu'))
student.add(Dropout(0.5))
student.add(Dense(2))
student.add(Activation('softmax'))

student.summary()

student.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 61, 61, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 57600)             0         
__________

/home/fukuda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,...)`
  if sys.path[0] == '':
/home/fukuda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  app.launch_new_instance()


In [92]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from xception import Xception

batch_size = 32
epoch = 10
rescale = 128
part ='eyes'

### setup dataset
data_dir = '../data/' +part
target_size = (rescale, rescale)

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    data_dir + '/validation',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')


### build teacher model
n_labels = train_generator.num_class
print('{} labels'.format(n_labels), train_generator.class_indices)

teacher = Xception(
    weights=None,
    classes=n_labels)
teacher.load_weights(
    'models/xception.rescale=128.eyes.weghit.h5')
teacher.summary()
teacher.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])


### build student model
student = Sequential()
student.add(Convolution2D(32, 3, 3, input_shape=(128, 128, 3)))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))

student.add(Convolution2D(64, 3, 3))
student.add(Activation('relu'))
student.add(MaxPooling2D(pool_size=(2, 2)))

student.add(Flatten())
student.add(Dense(64))
student.add(Activation('relu'))
student.add(Dropout(0.5))
student.add(Dense(2))
student.add(Activation('softmax'))

student.summary()

student.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])



Found 514 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
2 labels {'AB': 0, 'N': 1}
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0

/home/fukuda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,...)`
/home/fukuda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [94]:
from keras.layers import Activation, Dropout, Flatten, Dense, Add
from keras.models import Model

class EducateGenerator(object):
    def __init__(self, train_generator):
        self.train_generator = train_generator
        self.validation_generator = validation_generator
    def __next__(self):
        x_train, y_train = self.train_generator.next()
        return ( [x_train,x_train], y_train )
    def next(self):
        return self.__next__()

# .next() -> ([x_train,x_train],y_train)
educate_generator = EducateGenerator(
    train_generator)
    #, validation_generator)


### teacher educat student
negativeActivation = lambda x: -x

negativeRight = Activation(negativeActivation)(student.output)
diff = Add()([teacher.output,negativeRight])

model = Model(inputs=[teacher.input, student.input], outputs=[diff])
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['acc'])

model.summary(line_length=150)
#model.fit([X_train, X_train], [Y_train], batch_size=128, nb_epoch=5)
history = model.fit_generator(
    educate_generator,
    steps_per_epoch=200,#train_generator.n//batch_size,
    epochs=5)
#    validation_data=validation_generator,
#    validation_steps=20
#)

______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_2 (InputLayer)                             (None, None, None, 3)            0                                                                   
______________________________________________________________________________________________________________________________________________________
block1_conv1 (Conv2D)                            (None, None, None, 32)           864               input_2[0][0]                                     
______________________________________________________________________________________________________________________________________________________
block1_conv1_bn (BatchNormalization)             (None, None, None, 32)           128         

Epoch 1/5
  9/200 [>.............................] - ETA: 231s - loss: 0.5028 - acc: 0.6493

KeyboardInterrupt: 

In [2]:
import numpy as np
from keras_squeezenet import SqueezeNet
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image

model = SqueezeNet()

img = image.load_img('../images/cat.jpeg', target_size=(227, 227))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds))

TypeError: _obtain_input_shape() got an unexpected keyword argument 'include_top'

In [83]:
A = train_generator.next()
np.concatenate((A,A), axis=0)

ValueError: could not broadcast input array from shape (32,128,128,3) into shape (32)

In [80]:
educate_generator.next()

ValueError: could not broadcast input array from shape (32,128,128,3) into shape (32)

In [10]:

history = model.fit_generator(
    train_generator,
    samples_per_epoch=200,
    nb_epoch=epochs,
    validation_data=validation_generator,
    nb_val_samples=200)

/home/fukuda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/fukuda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=6, epochs=1, validation_steps=200)`
  import sys


Epoch 1/1
5/6 [========================>.....] - ETA: 1s - loss: 0.9498 - acc: 0.4562

KeyboardInterrupt: 

In [6]:
import os
result_dir = './models/'
model.save_weights(os.path.join(result_dir, 'smallcnn.h5'))
save_history(history, os.path.join(result_dir, 'history_smallcnn.txt'))

NameError: name 'os' is not defined

In [4]:
dir(train_generator

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_flow_index',
 'batch_index',
 'batch_size',
 'class_indices',
 'class_mode',
 'classes',
 'color_mode',
 'data_format',
 'directory',
 'filenames',
 'image_data_generator',
 'image_shape',
 'index_generator',
 'lock',
 'n',
 'next',
 'num_class',
 'reset',
 'samples',
 'save_format',
 'save_prefix',
 'save_to_dir',
 'shuffle',
 'target_size',
 'total_batches_seen']

In [205]:
history

import os

brain_dir = '../micin/brain/'
data_dir = '../data/brain/'

labels = ['N','MS','PD','PS']
for label in labels:
    try:
        os.mkdir(data_dir+label)
    except:
        pass


# 'PS116.jpg' -> ['PS', '116', '.jpg']
import itertools
strint_separator = lambda f:\
    [''.join(it) for _, it in itertools.groupby(f,str.isdigit)]
import os

brain_dir = '../micin/brain/'
data_dir = '../data/brain/'

labels = ['N','MS','PD','PS']
for label in labels:
    try:
        os.mkdir(data_dir+label)
    except:
        pass


# 'PS116.jpg' -> ['PS', '116', '.jpg']
import itertools
strint_separator = lambda f:\
    [''.join(it) for _, it in itertools.groupby(f,str.isdigit)]
flist = [ f for f in os.listdir(brain_dir) if '.jpg' in f ] # for removing .DS_Store
flist


for fname in flist:
    label = strint_separator(fname)[0] # 'N','MS','PD' or'PS'
    read_fname = brain_dir + fname
    write_fname = data_dir + label + '/' + fname
    print(read_fname, write_fname)
    #open(write_fname, 'wb

In [200]:
open('test.txt','w').write('eee')

3

In [121]:
# (pandas.DataFrame, ['N','AB']) -> [pandas.DataFrame]
# tv_rate : ratio of the data size of train vs validation 


In [52]:
train_generator = train_datagen.flow(
    x_train, y_train,
    target_size=(128, 128),
    batch_size=30,
    #class_mode='categorical'
)

TypeError: flow() got an unexpected keyword argument 'target_size'

In [43]:
num_classes = 10
import keras
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [5]:
import os

brain_dir = '../micin/brain/'
data_dir = '../data/brain/'

labels = ['N','MS','PD','PS']

try:
    os.mkdir(data_dir)
except:
    pass
for label in labels:
    try:
        os.mkdir(data_dir+label)
    except:
        pass


# 'PS116.jpg' -> ['PS', '116', '.jpg']
import itertools
strint_separator = lambda f:\
    [''.join(it) for _, it in itertools.groupby(f,str.isdigit)]


flist = [ f for f in os.listdir(brain_dir) if '.jpg' in f ] # for removing .DS_Store
for fname in flist:
    label = strint_separator(fname)[0] # 'N','MS','PD' or'PS'
    read_fname = brain_dir + fname
    write_fname = data_dir + label + '/' + fname
    open(write_fname, 'wb').write(open(read_fname,'rb').read())

../micin/brain/PS116.jpg ../data/brain/PS/PS116.jpg
../micin/brain/PD096.jpg ../data/brain/PD/PD096.jpg
../micin/brain/PD062.jpg ../data/brain/PD/PD062.jpg
../micin/brain/PD002.jpg ../data/brain/PD/PD002.jpg
../micin/brain/PD045.jpg ../data/brain/PD/PD045.jpg
../micin/brain/PS80.jpg ../data/brain/PS/PS80.jpg
../micin/brain/MS136.jpg ../data/brain/MS/MS136.jpg
../micin/brain/PD061.jpg ../data/brain/PD/PD061.jpg
../micin/brain/PS102.jpg ../data/brain/PS/PS102.jpg
../micin/brain/MS162.jpg ../data/brain/MS/MS162.jpg
../micin/brain/N070.jpg ../data/brain/N/N070.jpg
../micin/brain/PS018.jpg ../data/brain/PS/PS018.jpg
../micin/brain/PS118.jpg ../data/brain/PS/PS118.jpg
../micin/brain/PS041.jpg ../data/brain/PS/PS041.jpg
../micin/brain/PD046.jpg ../data/brain/PD/PD046.jpg
../micin/brain/PS028.jpg ../data/brain/PS/PS028.jpg
../micin/brain/PS112.jpg ../data/brain/PS/PS112.jpg
../micin/brain/PS031.jpg ../data/brain/PS/PS031.jpg
../micin/brain/PS050.jpg ../data/brain/PS/PS050.jpg
../micin/brain/PS

In [7]:
brain_dir = '../micin/brain/'
data_dir = '../data/brain'

labels = ['N','MS','PDbrain_dir = '../micin/brain/'
data_dir = '../data/brain'

labels = ['N','MS','PD','PS']
for label in labels:
    try:
        os.mkdir(data_dir+label)
    except:
        pass','PS']
for label in labels:
    try:
        os.mkdir(data_dir+label)
    except:
        pass

In [11]:
# 'PS116.jpg' -> ['PS', '116', '.jpg']
import itertools
strint_separator = lambda f:\
    [''.join(it) for _, it in itertools.groupby(f,str.isdigit)]

    

In [21]:
flist = [ f for f in os.listdir(brain_dir) if '.jpg' in f ] # for removing .DS_Store
for fname in flist: 
    label = strint_separator(fname)[0] # 'N','MS','PD' or'PS'
    read_fname = brain_dir + fname
    write_fname = data_dir + label + '/' + fname
    open(write_fname, 'wb').write(open(read_fname,'rb').read())